In [1]:
import pandas as pd
import difflib

# Venta

## Fase 1

Voy a dejar un poco más limpio el dataframe. Eliminaré variables que no aportan mucho (porque ya tienen un one-hot encoding, por ejemplo) y luego limpiaré aquellas observaciones que tienen `n_baths`=0.

In [2]:
df=pd.read_excel('data/pre-procesado/v-barcelones-abril24.xlsx')

Elimino `terrace_m2` porque no todos los pisos tienen información sobre su metraje. En cambio, sí sobre si tienen o no una terraza.

In [3]:
df=df.drop(['year_cons','floor','consum_EPC','emission_EPC','terrace_m2'],axis=1)

In [4]:
df = df[(df['n_baths'] != 0) & (df['area'] > 1)]
print("Longitud de datos:", len(df))

Longitud de datos: 16795


Ahora elimino duplicados.

In [5]:
# Función para calcular la similitud entre dos cadenas
def similarity(a, b):
    return difflib.SequenceMatcher(None, a, b).ratio()

# Umbral de similitud para 'Description'
description_threshold = 0.85

df['Description'] = df['Description'].astype(str)

# Eliminar duplicados en df
df['Description_Similarity'] = df['Description'].apply(lambda x: similarity(x, df['Description']))
duplicates_df = df[(df['Description_Similarity'] >= description_threshold)
                   &df.duplicated(subset=['area', 'precio_euros'], keep='first')]
df.drop(duplicates_df.index, inplace=True)

df.drop(columns=['Description_Similarity'], inplace=True)

# Verificar la longitud de los DataFrames resultantes
print("Longitud después de eliminar duplicados:", len(df))

IndentationError: unexpected indent (4102832922.py, line 13)

## Multifamiliares

In [ ]:
df = df[df['multifam'] == 1]
print("Longitud de df multifam:", len(df))

## Etiquetas EPC

Reviso cuántos pisos tienen información de etiquetas EPC.

In [ ]:
# Contar cuántas filas tienen valor 1 en la columna 'dummy_consum_EPC' de cada DataFrame
filas_con_1_consum_EPC_df = (df['dummy_consum_EPC'] == 1).sum()/len(df)*100

# Contar cuántas filas tienen valor 1 en la columna 'dummy_emission_EPC' de cada DataFrame
filas_con_1_emission_EPC_df = (df['dummy_emission_EPC'] == 1).sum()/len(df)*100

# Contar cuántas filas tienen valor 1 en ambas columnas 'dummy_consum_EPC' y 'dummy_emission_EPC' de cada DataFrame
filas_con_1_en_ambas_df = ((df['dummy_consum_EPC'] == 1) & (df['dummy_emission_EPC'] == 1)).sum()/len(df)*100

# Crear una tabla para mostrar el conteo por cada DataFrame
conteo_por_df = pd.DataFrame({
    'DataFrame': ['df'],
    'Total datos':[len(df)],
    'consum_EPC': [filas_con_1_consum_EPC_df],
    'emission_EPC': [filas_con_1_emission_EPC_df],
    'Ambas': [filas_con_1_en_ambas_df]
})

# Mostrar la tabla
print(conteo_por_df)

De los que tienen información, cuántos tienen valores inválidos.

In [ ]:
df['kwhm2_year'] = df['kwhm2_year'].fillna(0)
df['kgCO2m2_year'] = df['kgCO2m2_year'].fillna(0)

In [ ]:
# Calcular el porcentaje de filas donde 'kwhm2_year' tiene valor 999 o 0 (está vacía) respecto a 'dummy_consum_EPC' con valor 1 para cada DataFrame
porcentaje_kwhm2_999_consum_df = ((df['dummy_consum_EPC'] == 1) & (df['kwhm2_year'].isin([999, 0]))).mean() * 100

# Calcular el porcentaje de filas donde 'kgCO2m2_year' tiene valor 999 o 0 (está vacía) respecto a 'dummy_emission_EPC' con valor 1 para cada DataFrame
porcentaje_kgCO2m2_999_emission_df = ((df['dummy_emission_EPC'] == 1) & (df['kgCO2m2_year'].isin([999, 0]))).mean() * 100

# Crear una tabla para mostrar los porcentajes por cada DataFrame
porcentaje_por_df = pd.DataFrame({
    'DataFrame': ['df'],
    "999 consum": [porcentaje_kwhm2_999_consum_df],
    "999 emission": [porcentaje_kgCO2m2_999_emission_df]
})

# Mostrar la tabla
print(porcentaje_por_df)

<mark>¡Son un montón!</mark> Si las quitáramos, borraríamos todo el dataset, dado que ese 68% no necesariamente está dentro del 97% (ya lo he corroborado).

Por tanto, el código que tenía para eliminar aquellas filas con valores inválidos (999 o 0) en `kgCO2m2_year` o `kwhm2_year` ya no lo usaré, sólo limpiaré aquellas que no tienen datos de la etiqueta EPC.

In [ ]:
# Eliminar filas con valor 0 en 'dummy_consum_EPC' y/o 'dummy_emission_EPC'
df = df[~((df['dummy_consum_EPC'] == 0) | (df['dummy_emission_EPC'] == 0))]

# Mostrar la longitud de cada DataFrame después de eliminar las filas
print("Longitud después de eliminar filas:", len(df))

In [ ]:
# Eliminar filas con valores 999 o 0 en 'kgCO2m2_year' o 'kwhm2_year'
#df = df[~((df['kwhm2_year'].isin([999, 0])) | (df['kgCO2m2_year'].isin([999, 0])))]
#df.dropna(subset=['kwhm2_year', 'kgCO2m2_year'], inplace=True)

In [ ]:
df.to_excel('data/pre-procesado/v-barcelones-abril24-final.xlsx', index=False)

**Algo que podemos evaluar a futuro es centrarnos sólo en el consumo**, puesto que tendríamos una cantidad de observaciones relativamente parecida a la de años anteriores.

In [ ]:
len(df)*(100-porcentaje_kwhm2_999_consum_df)/100